## Use saprQL to query DBpedia

In [1]:
import rdflib
g=rdflib.Graph()
g.load('http://dbpedia.org/resource/Biomedical')

for s,p,o in g:
    print (s,p,o)

http://dbpedia.org/resource/Musa_Hakan_Asyalı http://dbpedia.org/ontology/knownFor http://dbpedia.org/resource/Biomedical
http://dbpedia.org/resource/Biomedical http://www.w3.org/2002/07/owl#sameAs http://dbpedia.org/resource/Biomedical
http://dbpedia.org/resource/Biomedical http://xmlns.com/foaf/0.1/isPrimaryTopicOf http://en.wikipedia.org/wiki/Biomedical
http://dbpedia.org/resource/Biomedical http://dbpedia.org/ontology/wikiPageID 609268
http://dbpedia.org/resource/Nadine_Barrie_Smith http://dbpedia.org/ontology/field http://dbpedia.org/resource/Biomedical
http://dbpedia.org/resource/Biomedical http://dbpedia.org/ontology/wikiPageRedirects http://dbpedia.org/resource/Medical_research
http://dbpedia.org/resource/Biomedical http://www.w3.org/ns/prov#wasDerivedFrom http://en.wikipedia.org/wiki/Biomedical?oldid=288855867
http://en.wikipedia.org/wiki/Biomedical http://xmlns.com/foaf/0.1/primaryTopic http://dbpedia.org/resource/Biomedical
http://dbpedia.org/resource/Charles_Stark_Draper_La

In [65]:
from SPARQLWrapper import SPARQLWrapper, JSON

prefix = """
    PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
    PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
    PREFIX dbpedia: <http://dbpedia.org/resource/>PREFIX dcterms: <http://purl.org/dc/terms/>
    PREFIX dbpedia-owl: <http://dbpedia.org/ontology/>PREFIX category: <http://dbpedia.org/resource/Category:>
    PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>PREFIX foaf: <http://xmlns.com/foaf/0.1/>PREFIX dbpprop: <http://dbpedia.org/property/>
    PREFIX dbprop: <http://dbpedia.org/property/>PREFIX grs: <http://www.georss.org/georss/>
    PREFIX category: <http://dbpedia.org/resource/Category:>
    PREFIX dcterms: <http://purl.org/dc/terms/>
    PREFIX owl: <http://www.w3.org/2002/07/owl#>
    PREFIX freebase: <http://rdf.freebase.com/ns/>
    PREFIX db: <http://dbpedia.org/>
    PREFIX dbp: <http://dbpedia.org/property/>
    PREFIX foaf: <http://xmlns.com/foaf/0.1/>
    PREFIX http: <http://www.w3.org/2006/http#>"""

In [14]:
sparql = SPARQLWrapper("http://dbpedia.org/sparql")
sparql.setQuery(prefix + """    
    SELECT ?z
    WHERE { <http://dbpedia.org/resource/Biomedical_engineering> rdf:type ?z }
""")
#Arye_Rosen
sparql.setReturnFormat(JSON)
results = sparql.query().convert()

for result in results["results"]["bindings"]:
    print(result)

{'z': {'type': 'uri', 'value': 'http://www.w3.org/2002/07/owl#Thing'}}
{'z': {'type': 'uri', 'value': 'http://xmlns.com/foaf/0.1/Person'}}
{'z': {'type': 'uri', 'value': 'http://dbpedia.org/ontology/Person'}}
{'z': {'type': 'uri', 'value': 'http://www.ontologydesignpatterns.org/ont/dul/DUL.owl#Agent'}}
{'z': {'type': 'uri', 'value': 'http://www.ontologydesignpatterns.org/ont/dul/DUL.owl#NaturalPerson'}}
{'z': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q215627'}}
{'z': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q24229398'}}
{'z': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q5'}}
{'z': {'type': 'uri', 'value': 'http://www.wikidata.org/entity/Q901'}}
{'z': {'type': 'uri', 'value': 'http://dbpedia.org/ontology/Agent'}}
{'z': {'type': 'uri', 'value': 'http://dbpedia.org/ontology/Scientist'}}
{'z': {'type': 'uri', 'value': 'http://schema.org/Person'}}


In [3]:
sparql = SPARQLWrapper("http://dbpedia.org/sparql")
sparql.setQuery(prefix + """
    SELECT ?z
    WHERE { <http://dbpedia.org/resource/Biomedical_engineering> rdfs:label ?z }
""")
sparql.setReturnFormat(JSON)
results = sparql.query().convert()

for result in results["results"]["bindings"]:
    if result['z']['xml:lang'] == 'en':
        print(result['z']['value'])

Biomedical engineering


## Search things in DBpedia and Store into mongoDB

In [4]:
from pymongo import MongoClient
client = MongoClient()
client = MongoClient('.amazonaws.com', 27017)
db = client.KG

In [74]:
#Replace symbol that may crash the query
def clean(word):
    word = word.replace(' ','_')
    word = word.replace('"','//')
    word = word.replace("'","/")
    return word


#Search the page title and get its pages
def get_type_from_db(title):
    sparql = SPARQLWrapper("http://dbpedia.org/sparql")
    sparql.setQuery(prefix + """    
        SELECT ?z
        WHERE { <http://dbpedia.org/resource/"""+title+"""> rdf:type ?z }
    """)
    sparql.setReturnFormat(JSON)
    results = sparql.query().convert()
    value = []
    for result in results["results"]["bindings"]:
        value.append(result['z']['value'].split('/')[-1])
    if 'Person' in value:
        return 'Person'
    elif 'University' in value:
        return 'University'



In [76]:
get_type_from_db(clean('Georgia_Institute_of_Technology'))

'University'

In [29]:

    

def get_info_from_db(title):
    page_type = get_type_from_db(title)



# Transfer pages in mongoDB to specific nodes with labels
def transfer_pages():
    for page in db.pages.find({}):
        page = get_info_from_db(clean(page['title'])
        #save_to_mongo(page)



SyntaxError: unexpected EOF while parsing (<ipython-input-29-d1b48129df46>, line 13)